<a href="https://colab.research.google.com/github/AlexisDaf/Project/blob/main/Troisi%C3%A8me%20partie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ======================
#  CONFIGURATION LORA
# ======================
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["c_attn"],  # spécifique à GPT-2
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)
model = get_peft_model(model, lora_config)

# ======================
#  TOKENISATION DU DATASET
# ======================
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["instruction", "input", "output", "text"])

# ======================
#  ARGUMENTS D'ENTRAÎNEMENT
# ======================
training_args = TrainingArguments(
    output_dir="./gpt2-lora-alpaca",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=20,
    save_steps=500,
    save_total_limit=2,
    fp16=True,
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

# ======================
#  ENTRAÎNEMENT
# ======================
trainer.train()